## Setup

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from mintpy.utils import readfile, writefile, utils as ut
from mintpy import view
import numpy as np
from pathlib import Path
import shutil

In [ ]:
cd /home/micromamba/data/spot/landslide_2_rangitikei

In [ ]:
loc = Path('mintpy')
if loc.exists():
    shutil.rmtree(loc)

loc.mkdir()

In [ ]:
cd mintpy

## Run MintPy

### Load Data

In [ ]:
rm ../corrected/*xml

In [ ]:
cp /home/micromamba/scripts/autorift.cfg ./autorift.cfg

In [ ]:
!load_data.py -t ./autorift.cfg

In [ ]:
az = readfile.read('inputs/ifgramStack.h5', datasetName='azimuthOffset')[0]
az_median = np.nanmedian(az, axis=(1,2))
az_new = az - az_median[:,None,None]

In [ ]:
rg = readfile.read('inputs/ifgramStack.h5', datasetName='rangeOffset')[0]
rg_median = np.nanmedian(rg, axis=(1,2))
rg_new = rg - rg_median[:,None,None]

In [ ]:
writefile.write({'azimuthOffset':az_new,'rangeOffset':rg_new},'inputs/ifgramStack_median.h5',ref_file='inputs/ifgramStack.h5')

## View stack

In [ ]:
view.main('inputs/ifgramStack.h5 azimuthOffset --noverbose'.split())

In [ ]:
view.main('inputs/ifgramStack.h5 rangeOffset --noverbose'.split())

## Inversion

In [ ]:
!ifgram_inversion.py inputs/ifgramStack.h5 -i rangeOffset -w no -o timeseriesRg.h5 residualInvRg.h5 numInvOffsetRg.h5
!ifgram_inversion.py inputs/ifgramStack.h5 -i azimuthOffset -w no -o timeseriesAz.h5 residualInvAz.h5 numInvOffsetAz.h5

In [ ]:
view.main('timeseriesAZ.h5 --noverbose'.split())

In [ ]:
view.main('timeseriesRg.h5 --noverbose'.split())

## timeseries2velocity

In [ ]:
!timeseries2velocity.py timeseriesAz.h5 -o velocityAz.h5 --bootstrap --bc 100
!timeseries2velocity.py timeseriesRg.h5 -o velocityRg.h5 --bootstrap --bc 100

## View Results

In [ ]:
view.main('velocityAz.h5 --vlim -500 500 --noverbose'.split())

In [ ]:
view.main('velocityRg.h5 --vlim -500 500 --noverbose'.split())

## Geocode

In [ ]:
!save_gdal.py velocityAz.h5 -o ../velocity_ns.tif
!save_gdal.py velocityRg.h5 -o ../velocity_ew.tif

## Done!